In [26]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [28]:
data=pd.read_csv("most_frequent_words_30.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
835,anime,help,95
836,anime,begin,94
837,anime,father,93
838,anime,attack,91


In [29]:
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
dict_words

{'western': ['town',
  'men',
  'take',
  'back',
  'one',
  'kill',
  'man',
  'ranch',
  'horse',
  'find',
  'ride',
  'killed',
  'go',
  'gang',
  'return',
  'two',
  'get',
  'gun',
  'sheriff',
  'tell',
  'indian',
  'shoot',
  'father',
  'brother',
  'help',
  'try',
  'woman',
  'gold',
  'new',
  'come'],
 'comedy': ['get',
  'find',
  'go',
  'one',
  'take',
  'tell',
  'friend',
  'back',
  'love',
  'new',
  'two',
  'father',
  'make',
  'life',
  'day',
  'home',
  'time',
  'family',
  'house',
  'also',
  'try',
  'come',
  'meet',
  'end',
  'see',
  'later',
  'man',
  'film',
  'money',
  'however'],
 'short': ['tom',
  'jerry',
  'bug',
  'stooge',
  'get',
  'back',
  'cat',
  'head',
  'spike',
  'curly',
  'mouse',
  'run',
  'go',
  'try',
  'one',
  'house',
  'end',
  'moe',
  'see',
  'away',
  'take',
  'make',
  'door',
  'two',
  'dog',
  'time',
  'chase',
  'fall',
  'sam',
  'find'],
 'action': ['kill',
  'one',
  'police',
  'get',
  'find',
  'ta

In [30]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [31]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Train[['Genre_updated','Plot_cleanned']][:10].apply(lambda x: reduce_words(x),axis=1)

Wall time: 15 ms


6     two two kill kill one try horse back tell men ...
7                               film family life go get
10                  get try get fall head back get fall
11                                            two later
12                                            two horse
13                                            two later
14                                             two make
15                                                     
16                                       return get get
17                    back home one friend house friend
dtype: object

In [32]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]
Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)

Wall time: 1min 8s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
Train.drop_duplicates(subset='Plot_trimmed',inplace=True)

<ipython-input-33-beb6f011043b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train.drop_duplicates(subset='Plot_trimmed',inplace=True)


In [34]:
Train.shape

(32334, 11)

In [35]:
%%time

X_train=Train['Plot_trimmed']
y_train=Train['Genre_updated']


X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.3,random_state=123,stratify=y_train)


X_test=Test['Plot_cleanned']
y_test=Test['Genre_updated']

Wall time: 29.9 ms


In [36]:
%%time
tfidf_vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,2))

X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 

X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) 


X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

Wall time: 2.29 s


In [37]:
X_train_vectors_tfidf.shape,X_val_vectors_tfidf.shape,X_test_vectors_tfidf.shape,

((22633, 8540), (9701, 8540), (6843, 8540))

In [38]:
text_clf = Pipeline([('tfidf', TfidfVectorizer(use_idf=True)),
                     ('clf', RandomForestClassifier(n_estimators = 100,random_state=123,n_jobs=-1))])
text_clf.fit(X_train,y_train)
#Predict y value for test dataset 
y_pred=text_clf.predict(X_val)
print(classification_report(y_val,y_pred))
print('Confusion Matrix:',confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

      action       0.84      0.90      0.87       671
   adventure       0.82      0.80      0.81       230
    animated       0.93      0.88      0.90       251
       anime       0.96      0.54      0.69        41
   biography       0.91      0.85      0.88       129
      comedy       0.80      0.82      0.81      1963
       crime       0.83      0.79      0.81       422
 documentary       0.88      0.41      0.56        37
       drama       0.79      0.92      0.85      2555
      family       0.92      0.75      0.83       178
     fantasy       0.94      0.72      0.82       152
  historical       0.93      0.62      0.74        81
      horror       0.93      0.89      0.91       440
martial_arts       1.00      0.74      0.85        38
     musical       0.92      0.75      0.83       255
     mystery       0.86      0.62      0.72       129
        noir       0.93      0.54      0.68        98
     romance       0.95    

In [43]:
data=Train[['Genre_updated','Plot_trimmed']]
data

,Genre_updated,Plot_trimmed
6,western,two two kill kill one try horse back tell men ...
7,comedy,film family life go get
10,short,get try get fall head back get fall
11,action,two later
12,western,two horse
...,...,...
42525,drama,film two take family go find tell wife family ...
42526,drama,family mother find new
42532,comedy,father also friend try family find
42533,comedy,film however two go man go man new life get life
